# dcm -> nii 변환

## 주의사항
: 이미지와 affine matrix를 맞춰줘야함 (LPS, RAS 중 동일한 좌표계를 표시하도록)

- 원본 dcm 파일 (x y z) = LPS

- pydicom으로 읽은 파일 (z y x) = x

- transpose(2,1,0) 적용 (x y z) = LPS

- flip(axis=0), flip(axis=1) 적용 = RAS


### flip 안한 축은, affine에 - 붙이기

nii는 기본적으로 RAS 사용하기에

In [ ]:
''' import '''
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import pydicom
import nibabel as nib

from dicom import dcmread_volume

In [ ]:
root_path = os.path.join(os.getcwd(), '..', '..',  'dicom_matlab', 'data', 'patient-example')

ct_folder = None
rtdose_folder = None
rtst_folder = None

ct_files = []
rtdose_files = []
rtst_files = []

# folders
for folder in os.listdir(root_path):
    if 'CT' in folder:
        ct_folder = os.path.join(root_path, folder)
    if 'RTDOSE' in folder:
        rtdose_folder = os.path.join(root_path, folder)
    if 'RTst' in folder:
        rtst_folder = os.path.join(root_path, folder)

# files
for file in os.listdir(ct_folder): # CT
    if file.endswith('.dcm'):
        ct_files.append(os.path.join(ct_folder, file))

for file in os.listdir(rtdose_folder): # RT dose
    if file.endswith('.dcm'):
        rtdose_files.append(os.path.join(rtdose_folder, file))

for file in os.listdir(rtdose_folder): # RT structure
    if file.endswith('.dcm'):
        rtst_files.append(os.path.join(rtst_folder, file))

# file
rtdose_file = rtdose_files[0]
rtst_file = rtst_files[0]

# CT

In [ ]:
img_offset = np.zeros(3, dtype=np.double)
img_orie = np.zeros((2,3), dtype=np.int16)
img_spacing = np.zeros(3, dtype=np.double)
img_size = np.zeros(3, dtype=np.int16)


''' CT meta '''
ct_info_tmp = pydicom.dcmread(ct_files[0])

rescale_slope = ct_info_tmp.RescaleSlope
rescale_intercept = ct_info_tmp.RescaleIntercept


''' CT img '''
image_raw, spatial = dcmread_volume(ct_folder)

image = (image_raw*rescale_slope) + rescale_intercept

img_offset[:] = spatial.PatientPositions[0, :]
img_orie[:] = spatial.PatientOrientations[0, :]
img_spacing[:2] = spatial.PixelSpacings[0, :]
img_spacing[2] = spatial.PatientPositions[1, 2] - spatial.PatientPositions[0, 2]
img_size[:] = spatial.ImageSize # 337 512 512         # matlab : 512(y=r) 512(x=c) 337

In [ ]:
# '''
#     dcm2niix 사용 (ITK : LAS)     LPS + A-P반전
# '''
# import subprocess

# root_path = os.path.join(os.getcwd(), '..', '..',  'dicom_matlab', 'data', 'patient-example')

# input_path = os.path.join(root_path, 'DOE^JANE_ANON76206_CT_2021-05-27_171730_._reduceCT_n337__00000')
# output_path = os.path.join(root_path, '..')

# subprocess.run(['dcm2niix', '-z', 'y', '-o', output_path, '-f', 'ct_d2n', input_path])

In [ ]:
'''
    dcm2niix 모방 (ITK : LAS)
'''
image_d2n = np.transpose(image, (2,1,0))
# image_d2n = np.flip(image_d2n, axis=0)
image_d2n = np.flip(image_d2n, axis=1) # (P->A)

# affine matrix
x_off, y_off, z_off = img_offset
x_spc, y_spc, z_spc = img_spacing
x_ori = img_orie[0,:]
y_ori = img_orie[1,:]
z_ori = np.cross(img_orie[0,:], img_orie[1,:])

ct_affine_d2n = np.array([
    [-x_ori[0]*x_spc, -y_ori[0]*y_spc, -z_ori[0]*z_spc, -x_off],
    [x_ori[1]*x_spc, y_ori[1]*y_spc, z_ori[1]*z_spc, y_off],
    [x_ori[2]*x_spc, y_ori[2]*y_spc, z_ori[2]*z_spc, z_off],
    [0, 0, 0, 1]
])

ct_d2n_nii = nib.Nifti1Image(image_d2n, affine=ct_affine_d2n)

nib.save(ct_d2n_nii, '../data/ct_d2n_copy.nii.gz')

In [ ]:
''' LPS (ITK : LPS) '''
lps = np.transpose(image, (2,1,0))
# lps = np.flip(lps, axis=0)
# lps = np.flip(lps, axis=1)

# affine matrix
x_off, y_off, z_off = img_offset
x_spc, y_spc, z_spc = img_spacing
x_ori = img_orie[0,:]
y_ori = img_orie[1,:]
z_ori = np.cross(img_orie[0,:], img_orie[1,:])

ct_affine_d2n = np.array([
    [-x_ori[0]*x_spc, -y_ori[0]*y_spc, -z_ori[0]*z_spc, -x_off],
    [-x_ori[1]*x_spc, -y_ori[1]*y_spc, -z_ori[1]*z_spc, -y_off],
    [x_ori[2]*x_spc, y_ori[2]*y_spc, z_ori[2]*z_spc, z_off],
    [0, 0, 0, 1]
])

ct_lps = nib.Nifti1Image(lps, affine=ct_affine_d2n)

nib.save(ct_lps, '../data/ct_lps.nii.gz')

In [ ]:
''' RAS (ITK : RAS) '''
ras = np.transpose(image, (2,1,0))
ras = np.flip(ras, axis=0) # LPS -> RAS
ras = np.flip(ras, axis=1)

# affine matrix
x_off, y_off, z_off = img_offset
x_spc, y_spc, z_spc = img_spacing
x_ori = img_orie[0,:]
y_ori = img_orie[1,:]
z_ori = np.cross(img_orie[0,:], img_orie[1,:])

ct_affine_d2n = np.array([
    [x_ori[0]*x_spc, y_ori[0]*y_spc, z_ori[0]*z_spc, x_off],
    [x_ori[1]*x_spc, y_ori[1]*y_spc, z_ori[1]*z_spc, y_off],
    [x_ori[2]*x_spc, y_ori[2]*y_spc, z_ori[2]*z_spc, z_off],
    [0, 0, 0, 1]
])

ct_ras = nib.Nifti1Image(ras, affine=ct_affine_d2n)

nib.save(ct_ras, '../data/ct_ras.nii.gz')

In [ ]:
''' 
    affine matrix만 변경 (ITK : SPL) = xyz가 바뀌어 출력
'''

# affine matrix
x_off, y_off, z_off = img_offset
x_spc, y_spc, z_spc = img_spacing
x_ori = img_orie[0,:] # LPS -> RAS (x,y 반전)
y_ori = img_orie[1,:]
z_ori = np.cross(img_orie[0,:], img_orie[1,:])

ct_affine_aff = np.array([
    [-z_ori[0]*z_spc, -y_ori[0]*y_spc, -x_ori[0]*x_spc, -x_off],
    [-z_ori[1]*z_spc, -y_ori[1]*y_spc, -x_ori[1]*x_spc, -y_off],
    [z_ori[2]*z_spc, y_ori[2]*y_spc, x_ori[2]*x_spc, z_off],
    [0, 0, 0, 1]
])

ct_nii_aff = nib.Nifti1Image(image, affine=ct_affine_aff)

nib.save(ct_nii_aff, '../data/ct_aff.nii.gz')

In [ ]:
''' read nii '''
data = nib.load('../data/ct_d2n.nii.gz')

info = data.header
img = data.get_fdata()
# print(info)

In [ ]:
# plt.imshow(img[250,:,:])
# plt.imshow(img[:,250,:])
# plt.imshow(img[:,:,160])

In [ ]:
# # x
# for i in range(round(img_size[0]/3)):
#     clear_output(wait=True)
#     plt.imshow(image[i*3,:,:])
#     plt.show()

# # y
# for i in range(round(img_size[1]/3)):
#     clear_output(wait=True)
#     plt.imshow(image[:,i*3,:])
#     plt.show()

# # z
# for i in range(round(img_size[2]/3)):
#     clear_output(wait=True)
#     plt.imshow(image[:,:,i*3].T)
#     plt.show()

# RT dose
data 구조는 CT와 동일(z y x, LPS)